### RGCN 
In this notebook we want to make an easy implementation of the RGCN model from scratch - namely using only Pytorch (no pyg or dgl or whatever)

The first task will probably be defining a class to define a RGCN layer. 

In order to do that we will have to remember what we need!

- 

In [53]:
#import libraries

import torch 
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import kgbench as kg
import fire, sys

from kgbench import load, tic, toc, d, Data


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors


#
from torch_geometric.utils import to_networkx
import networkx as nx


In [36]:
data = kg.load('aifb', torch=True) 
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r
data.triples


loaded data aifb (0.4663s).
Number of entities: 8285
Number of classes: 4
Types of relations: 45


tensor([[4938,   39, 8181],
        [4938,   42, 8175],
        [4938,   44, 5292],
        ...,
        [7628,   37,  350],
        [7628,   39, 5220],
        [8052,   39, 8178]])

In [55]:
class MyCustomObject(Data):
    def __init__(self, data):
        self.data = data
        
    def to(self, device):
        # move all tensor attributes to the specified device
        for name, attr in self.__dict__.items():
            if isinstance(attr, torch.Tensor):
                setattr(self, name, attr.to(device))
        
        return self

In [58]:
data = MyCustomObject(data)
data.to(torch.device('cpu'))

In [59]:
def to(data, device):
    # move all tensor attributes to the specified device
    for name, attr in data.__dict__.items():
        if isinstance(attr, torch.Tensor):
            setattr( name, attr.to(device))

In [ ]:
to(data)

In [49]:
data.edge_type = torch.tensor(data.triples[:, 1])
#[data.i2r[i] for i in data.triples[:, 1]]
data.edge_type

/var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/ipykernel_74074/3187734064.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.edge_type = torch.tensor(data.triples[:, 1])


tensor([39, 42, 44,  ..., 37, 39, 39])

In [52]:
data.num_relations

45

In [50]:
idxt, clst = data.training[:, 0], data.training[:, 1]
idxw, clsw = data.withheld[:, 0], data.withheld[:, 1]
data.train_idx, data.train_y = idxt.long(), clst.long()
data.test_idx, data.test_y = idxw.long(), clsw.long()
data.edge_index = torch.stack((data.triples[:, 0], data.triples[:, 2]),dim=0)
data.edge_type = torch.tensor(data.triples[:, 1])

/var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/ipykernel_74074/889591701.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.edge_type = torch.tensor(data.triples[:, 1])


In [30]:
from numpy import dtype


data.triples = torch.tensor(data.triples, dtype=torch.int32)
data.triples

/var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/ipykernel_74074/943310574.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.triples = torch.tensor(data.triples, dtype=torch.int32)


tensor([[1500253,       0, 1466391],
        [1500253,       3, 1498119],
        [1500253,       3, 2218766],
        ...,
        [2210166,      10, 1316458],
        [2210166,     226,  607186],
        [2210166,     227,  232893]], dtype=torch.int32)

In [60]:
data = torch.load("IMDb_typePeople_data.pt")
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r
data.triples

Number of entities: 2420556
Number of classes: 19
Types of relations: 228


array([[1500253,       0, 1466391],
       [1500253,       3, 1498119],
       [1500253,       3, 2218766],
       ...,
       [2210166,      10, 1316458],
       [2210166,     226,  607186],
       [2210166,     227,  232893]])

In [64]:
idxt, clst = torch.Tensor(data.training[:, 0]), torch.Tensor(data.training[:, 1])
idxw, clsw = torch.Tensor(data.withheld[:, 0]), torch.Tensor(data.withheld[:, 1])
data.train_idx, data.train_y = idxt.long(), clst.long()
data.test_idx, data.test_y = idxw.long(), clsw.long()
data.triples = torch.tensor(data.triples)
data.edge_index = torch.stack((data.triples[:, 0], data.triples[:, 2]),dim=0)
data.edge_type = torch.tensor(data.triples[:, 1])

/var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/ipykernel_74074/2351473034.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.triples = torch.tensor(data.triples)
/var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/ipykernel_74074/2351473034.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.edge_type = torch.tensor(data.triples[:, 1])


In [63]:
data.edge_index

tensor([[1500253, 1500253, 1500253,  ..., 2210166, 2210166, 2210166],
        [1466391, 1498119, 2218766,  ..., 1316458,  607186,  232893]],
       dtype=torch.int32)

In [173]:
from rgcn_explainer_utils import *

data = kg.load('aifb', torch=True, final=True) 
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r
data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
a = get_relations(data)


loaded data aifb (0.2708s).
Number of entities: 8285
Number of classes: 4
Types of relations: 45


/usr/local/Caskroom/miniconda/base/lib/python3.9/site-packages/kgbench/load.py:103: UserWarning: The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, the correct evaluation is to combine train and validation for the final evaluation run.Set include_val to True when loading the data.
  warnings.warn('The validation data is not added to the training data. For AIFB, MUTAG, BGS and AM, '


In [186]:
for i in data.triples:
    if i[:2][1]==2:
        print(i)

tensor([6813,    2, 5791])
tensor([6813,    2, 5861])
tensor([6813,    2, 5876])
tensor([6813,    2, 5886])
tensor([6813,    2, 5897])
tensor([6814,    2, 5698])
tensor([6814,    2, 5704])
tensor([6814,    2, 5759])
tensor([6815,    2, 5900])
tensor([6815,    2, 5993])
tensor([6815,    2, 6543])
tensor([6816,    2, 5850])
tensor([6816,    2, 5867])
tensor([6816,    2, 5903])
tensor([6817,    2, 5740])
tensor([6817,    2, 5775])
tensor([6817,    2, 5791])
tensor([6817,    2, 5895])
tensor([6817,    2, 5907])
tensor([6818,    2, 5912])
tensor([6818,    2, 6462])
tensor([6818,    2, 6531])
tensor([6818,    2, 6532])
tensor([6820,    2, 5740])
tensor([6820,    2, 5799])
tensor([6821,    2, 5740])
tensor([6821,    2, 5755])
tensor([6822,    2, 5791])
tensor([6822,    2, 5876])
tensor([6822,    2, 6545])
tensor([6822,    2, 6618])
tensor([6823,    2, 5779])
tensor([6823,    2, 6412])
tensor([6823,    2, 6489])
tensor([6823,    2, 6515])
tensor([6823,    2, 6518])
tensor([6825,    2, 5772])
t

In [189]:
data.i2e[5720]

('http://www.aifb.uni-karlsruhe.de/Personen/viewPersonOWL/id1960instance',
 'iri')

In [168]:
data.edge_index = edge_index_oneadj(data.triples)

index, neighbors, index_t = find_n_hop_neighbors(data.edge_index, 3, 5698)

In [169]:
len(neighbors)

4890

[(5355, 5970),
 (5355, 5986),
 (5355, 5698),
 (5366, 5698),
 (5372, 5698),
 (5378, 5984),
 (5378, 5986),
 (5378, 5698),
 (5453, 5698),
 (5478, 5984),
 (5478, 5698),
 (5493, 5698),
 (5494, 5941),
 (5494, 5986),
 (5494, 5698),
 (5502, 5941),
 (5502, 5970),
 (5502, 5984),
 (5502, 5986),
 (5502, 5698),
 (5502, 6814),
 (5502, 6841),
 (5502, 6843),
 (5502, 6844),
 (5502, 6846),
 (5502, 6890),
 (5502, 7002),
 (5502, 7007),
 (5502, 7052),
 (5502, 7078),
 (5502, 7079),
 (5502, 7080),
 (5502, 7085),
 (5502, 7093),
 (5502, 7095),
 (5502, 7110),
 (5502, 7111),
 (5502, 7379),
 (5502, 7429),
 (5502, 7430),
 (5502, 7431),
 (5502, 7432),
 (5502, 7439),
 (5502, 7440),
 (5502, 7442),
 (5502, 7443),
 (5502, 7452),
 (5502, 7456),
 (5502, 7458),
 (5502, 7459),
 (5502, 7477),
 (5502, 7479),
 (5502, 7613),
 (5502, 7614),
 (5502, 7620),
 (5502, 7733),
 (5502, 7734),
 (5502, 7798),
 (5502, 7800),
 (5502, 7801),
 (5502, 7957),
 (5502, 7959),
 (5502, 7961),
 (5502, 7991),
 (5502, 7992),
 (5502, 7994),
 (5502, 79

In [144]:
%cd /Users/macoftraopia/Documents/GitHub/RGCN-Explainer
mislabeled = []
correct = []
for i in range(data.withheld.shape[0]):
    if torch.load('aifb_chk/prediction_aifb').argmax(dim=1)[i] != data.withheld[i][1]:
        mislabeled.append(data.withheld[i][0])
    else:
        correct.append(data.withheld[i][0])

/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


[tensor(5880, dtype=torch.int32), tensor(5898, dtype=torch.int32)]

In [148]:
data.withheld[:, 0].long()

tensor([5878, 5858, 5774, 5696, 5853, 5728, 5698, 5837, 5725, 5776, 5747, 5880,
        5899, 5852, 5687, 5819, 5807, 5787, 5773, 5856, 5824, 5862, 5895, 5898,
        5881, 5876, 5679, 5863, 5833, 5693, 5869, 5788, 5809, 5806, 5768])

In [129]:
%cd /Users/macoftraopia/Documents/GitHub/RGCN-Explainer
a = torch.load('aifb_chk/prediction_aifb')
a.argmax(dim=1)[6]


/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


tensor(0)

In [132]:
data.withheld[i][1]
data.withheld.shape[0]

35

In [171]:


relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]
relations = ['label', 'node_idx'] + relations
relations

AttributeError: 'Data' object has no attribute 'i2rel'

In [98]:
node_idx = 5757
df = pd.DataFrame(columns=relations)
h = {'isWorkedOnBy': 11, 'author': 11, 'isAbout': 8, 'member': 2, 'dealtWithIn': 1, 'fax': 1, 'phone': 1, 'name': 1, 'homepage': 1, '1999': 1, 'photo': 1, 'projectInfo': 1, 'hasProject': 1}
info = {'label':0, 'node_idx': str(node_idx)}
h.update(info)
df.loc[str(node_idx)] = h
df.loc[str(2)] = h
df.to_csv('most_important_relations.csv', index=False)


In [102]:
d = {key.item(): data.withheld[:, 0][data.withheld[:, 1] == key].tolist() for key in torch.unique(data.withheld[:, 1])}
d

{0: [5678,
  5724,
  5699,
  5688,
  5702,
  5714,
  5708,
  5843,
  5873,
  5697,
  5783,
  5701,
  5845,
  5778],
 1: [5731, 5905, 5808, 5785],
 2: [5757, 5797, 5900, 5677, 5791, 5811, 5831, 5839, 5755, 5844, 5861],
 3: [5857, 5752, 5795, 5753, 5798, 5854]}

In [106]:
data.i2r[5]

'http://swrc.ontoware.org/ontology#carriesOut'

In [90]:
h = {'isWorkedOnBy': 11, 'author': 11, 'isAbout': 8, 'member': 2, 'dealtWithIn': 1, 'fax': 1, 'phone': 1, 'name': 1, 'homepage': 1, '1999': 1, 'photo': 1, 'projectInfo': 1, 'hasProject': 1}
info = {'label':0, 'node_idx': str(node_idx)}
h.update(info)
h

{'isWorkedOnBy': 11,
 'author': 11,
 'isAbout': 8,
 'member': 2,
 'dealtWithIn': 1,
 'fax': 1,
 'phone': 1,
 'name': 1,
 'homepage': 1,
 '1999': 1,
 'photo': 1,
 'projectInfo': 1,
 'hasProject': 1,
 'label': 0,
 'node_idx': '5757'}

In [3]:
#one adjacency 
def edge_index_oneadj(data):
    edge_index = torch.stack((data.triples[:, 0], data.triples[:, 2]),dim=0)
    return edge_index


def edge_index_reladj(data):
    data.edge_index = torch.stack((data.triples[:, 0], data.triples[:, 2]), dim=0)
    data.edge_type = data.triples[:, 1]
    return data.edge_index, data.edge_type


def edge_index_dic(data):# Create an empty dictionary
    data.edge_index = {}

    # Iterate over some keys and values and append the values to the dictionary
    for i in np.unique(data.triples[:,1]):
        # Create some key and value (e.g. index and data)
        key = i
        value = [(s.item(), o.item()) for s, p, o in data.triples[torch.eq(data.triples[:, 1], i)]]
        value = torch.Tensor(value).t().long()
        #value = torch.Tensor(value).long()
        # Check if the key is already in the dictionary
        if key in data.edge_index:
            # If the key is already in the dictionary, append the value to the existing list
            data.edge_index[key].append(value)
        else:
            # If the key is not yet in the dictionary, create a new list with the value
            data.edge_index[key] = value
    return data.edge_index   

def get_adjacency(data):
    data.edge_index = edge_index_oneadj(data)
    adj = torch.zeros(data.num_entities, data.num_entities)
    for edge in data.edge_index.t():
        adj[edge[0]][edge[1]] = 1
        adj[edge[1]][edge[0]] = 1

    return adj



def get_adjacencies_dict(data):
    data.edge_index = edge_index_dic(data)
    adj = torch.zeros(data.num_entities, data.num_entities)
    for i in np.unique(data.triples[:,1]):
        for edge in data.edge_index[i].t():
            adj[edge[0]][edge[1]] = i+1
            adj[edge[1]][edge[0]] = i+1

    
    return adj

In [ ]:
# get_adjacencies_dict(data)
# get_adjacency(data)

## Nieghborhood
We define functions to extract neighborhood for the sake of visualization of node connectivity (and because it will be somehow useful then)

In [4]:
def neighborhoods(adj, n_hops):
    """Returns the n_hops degree adjacency matrix adj."""
    adj = torch.tensor(adj, dtype=torch.float)
    hop_adj = power_adj = adj
    for i in range(n_hops - 1):
        power_adj = power_adj @ adj
        prev_hop_adj = hop_adj
        hop_adj = hop_adj + power_adj
        #print(type(hop_adj))
        hop_adj = (hop_adj > 0).float()
        #print(hop_adj)
    return hop_adj.cpu().numpy().astype(int)

def extract_neighborhood(node_idx,adj,n_hops):
    """Returns the neighborhood of a given ndoe."""
    neighbors_adj_row = neighborhoods(adj,n_hops)[node_idx, :] #take row of the node in the new adj matrix
    # index of the query node in the new adj
    node_idx_new = sum(neighbors_adj_row[:node_idx]) #sum of all the nodes before the query node (since they are 1 or 0) - it becomes count of nodes before the query node
    neighbors = np.nonzero(neighbors_adj_row)[0] #return the indices of the nodes that are connected to the query node (and thus are non zero)
    sub_adj = adj[neighbors][:, neighbors]
    return node_idx_new, neighbors , sub_adj

#node_idx_new, neighbors, sub_adj = extract_neighborhood(5757,get_adjacency(data),1)
node_idx_new, neighbors, sub_adj = extract_neighborhood(5757,get_adjacencies_dict(data),1)

/var/folders/ww/33zq_rh50tx94n81lb4thx0w0000gn/T/ipykernel_72369/4292683132.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  adj = torch.tensor(adj, dtype=torch.float)


In [ ]:
get_adjacencies_dict(data)

In [ ]:
#MICRO DATASET
adj_dict = get_adjacencies_dict(data)
#adj_dict = sub_adj

G = nx.from_pandas_adjacency(pd.DataFrame(adj_dict.detach().numpy()))

colors = [adj_dict.detach().numpy()[u][v] for u, v in G.edges()]

nx.draw(G, with_labels=True, edge_color=colors)

In [ ]:
#AIFB
adj_dict = sub_adj

G = nx.from_pandas_adjacency(pd.DataFrame(adj_dict.detach().numpy()))

colors = [adj_dict.detach().numpy()[u][v] for u, v in G.edges()]

nx.draw(G, with_labels=True, edge_color=colors)

In [ ]:
adj_dict

### Data preparation
- add self-loops and inverse relation to the KG
- adjacency matrix for each relation type



In [ ]:
def enrich(triples : torch.Tensor, n : int, r: int):
    """
    Enriches the given triples with self-loops and inverse relations.

    """
    cuda = triples.is_cuda

    inverses = torch.cat([
        triples[:, 2:],
        triples[:, 1:2] + r,
        triples[:, :1]
    ], dim=1)

    selfloops = torch.cat([
        torch.arange(n, dtype=torch.long,  device=d(cuda))[:, None],
        torch.full((n, 1), fill_value=2*r),
        torch.arange(n, dtype=torch.long, device=d(cuda))[:, None],
    ], dim=1)

    return torch.cat([triples, inverses, selfloops], dim=0)

a = enrich(data.triples, data.num_entities, data.num_relations)
len(a)

## ADJ
The whole idea of creating the adjacency matrix for multiple relation in here is based on the idea of having an adjacency matrix with size :(num_nodes * num_nodes,num_nodes * num_nodes)  where in each portion of the adjacency matrix a different relation is defined.

So for instance in the first 'quadrant' nxn we have the relations of relation type 1, in the second 'quadrant' relation type 2 and so on - where to move between quadrants we 'jump' of num_nodes rows and columns.

In [ ]:
#here we print the adjacency matrix with original triples and where the different relation types are not taken into account
#original adjacency - no enriched triples
n = data.num_entities
adj_matrix = np.zeros((n, n), dtype=int)

# Fill the adjacency matrix with the edges
for edge in edge_index_oneadj(data).t():
#for edge in hor_ind:
    adj_matrix[edge[0]][edge[1]] = 1
    #adj_matrix[edge[1]][edge[0]] = 1

print(adj_matrix)

In [ ]:
def adj(triples, num_nodes, num_rels, cuda=False, vertical=True):
    """
     Computes a sparse adjacency matrix for the given graph (the adjacency matrices of all
     relations are stacked vertically).

     :param edges: List representing the triples
     :param i2r: list of relations
     :param i2n: list of nodes
     :return: sparse tensor
    """
    r, n = num_rels, num_nodes
    size = (r * n, n) if vertical else (n, r * n)

    from_indices = []
    upto_indices = []

    for s, p, o in triples:

        offset = p.item() * n
        print(offset)

        if vertical:
            s = offset + s.item()
        else:
            o = offset + o.item()

        from_indices.append(s)
        upto_indices.append(o)
        

    indices = torch.tensor([from_indices, upto_indices], dtype=torch.long, device=d(cuda))


    assert indices.size(1) == len(triples)
    assert indices[0, :].max() < size[0], f'{indices[0, :].max()}, {size}, {r}'
    assert indices[1, :].max() < size[1], f'{indices[1, :].max()}, {size}, {r}'

    return indices.t(), size
triples = enrich(data.triples,data.num_entities, data.num_relations)
indices, size = adj(triples, data.num_entities, triples.shape[0], cuda=False, vertical=False)

#indices, size = adj(data.triples, data.num_entities, data.num_relations, cuda=False, vertical=True)
# print(indices)
# print(data.triples)

def get_adjacency(indices):
    #data.edge_index = edge_index_oneadj(data)
    adj = torch.zeros(size[0]*4, size[0]*4)
    for edge in indices.t():
        print(edge)
        adj[edge[0]][edge[1]] = 1
        adj[edge[1]][edge[0]] = 1
        #adj[edge[0]][edge[0]] = 1

    return adj

adj_matrix = get_adjacency(indices)
print(adj_matrix)
size

In [ ]:
def neighborhoods(adj, n_hops):
    """Returns the n_hops degree adjacency matrix adj."""
    adj = torch.tensor(adj, dtype=torch.float)
    hop_adj = power_adj = adj
    for i in range(n_hops - 1):
        power_adj = power_adj @ adj
        prev_hop_adj = hop_adj
        hop_adj = hop_adj + power_adj
        #print(type(hop_adj))
        hop_adj = (hop_adj > 0).float()
        #print(hop_adj)
    return hop_adj.cpu().numpy().astype(int)

def extract_neighborhood(node_idx,adj,n_hops):
    """Returns the neighborhood of a given ndoe."""
    neighbors_adj_row = neighborhoods(adj,n_hops)[node_idx, :] #take row of the node in the new adj matrix
    # index of the query node in the new adj
    node_idx_new = sum(neighbors_adj_row[:node_idx]) #sum of all the nodes before the query node (since they are 1 or 0) - it becomes count of nodes before the query node
    neighbors = np.nonzero(neighbors_adj_row)[0] 
    for i in range(len(neighbors)):
        if neighbors[i] > data.num_entities:
            neighbors[i] = neighbors[i] - data.num_entities#return the indices of the nodes that are connected to the query node (and thus are non zero)
    sub_adj = adj[neighbors][:, neighbors]
    return node_idx_new, neighbors , sub_adj

In [ ]:
#a = get_adjacency(data)
print(neighborhoods(adj_matrix,2))
# print(neighborhoods(a,1))
extract_neighborhood(1,adj_matrix,2)


In [ ]:
#Here we show the adjacency matrix of the original triples where the different relation types are taken into account
print('original triples:', data.triples)


n = data.num_entities*2
hor_graph = np.zeros((n, n), dtype=int)
ver_graph =  np.zeros((n, n), dtype=int)
hor_ind, hor_size = adj(data.triples, data.num_entities, data.num_relations, cuda=False, vertical=False)
ver_ind, ver_size = adj(data.triples, data.num_entities, data.num_relations, cuda=False, vertical=True)

for edge in hor_ind:
    hor_graph[edge[0]][edge[1]] = 1
    #adj_matrix[edge[1]][edge[0]] = 1
    
for edge in ver_ind:
    ver_graph[edge[0]][edge[1]] = 1
#print(adj_matrix)
#print(hor_ind)
print('relation 0 adj matrix: \n',hor_graph[0:5][:, 0:5])
print('relation 1 adj matrix: \n',hor_graph[0:5][:,5:10])


In [ ]:
def sum_sparse(indices, values, size, row=True):
    """
    Sum the rows or columns of a sparse matrix, and redistribute the
    results back to the non-sparse row/column entries

    :return:
    """

    ST = torch.cuda.sparse.FloatTensor if indices.is_cuda else torch.sparse.FloatTensor

    assert len(indices.size()) == 2

    k, r = indices.size()

    if not row:
        # transpose the matrix
        indices = torch.cat([indices[:, 1:2], indices[:, 0:1]], dim=1)
        size = size[1], size[0]

    ones = torch.ones((size[1], 1), device=d(indices))

    smatrix = ST(indices.t(), values, size=size)
    sums = torch.mm(smatrix, ones) # row/column sums

    sums = sums[indices[:, 0]]

    assert sums.size() == (k, 1)

    return sums.view(k)

#sum_sparse(indices, ,size, row=True)

## Define the RGCN class module

We do:


In [ ]:
#triples = enrich(data.triples, data.num_entities, data.num_relations)
triples = data.triples
hor_ind, hor_size = adj(triples, data.num_entities, 2*data.num_relations+1, cuda=False, vertical=False)
ver_ind, ver_size = adj(triples, data.num_entities, 2*data.num_relations+1, cuda=False, vertical=True)
_, rn = hor_size #horizontally stacked adjacency matrix size
r = rn // data.num_entities 
print('r:', r)
print('rn:', rn)

vals = torch.ones(ver_ind.size(0), dtype=torch.float) #number of edges
vals = vals / sum_sparse(ver_ind, vals, ver_size)
print('vals is', vals)

hor_graph = torch.sparse.FloatTensor(indices=hor_ind.t(), values=vals, size=hor_size)
print('hor graph is: ', hor_graph)

ver_graph = torch.sparse.FloatTensor(indices=ver_ind.t(), values=vals, size=ver_size)
print('ver graph is: ', ver_graph)
# weights1 = nn.Parameter(torch.FloatTensor(r, data.num_entities, 16))
# print('weights1 is: ', weights1, 'shape is: ', weights1.shape)

In [ ]:
n, r = data.num_entities, data.num_relations
ind,size = adj(enrich(data.triples,n,r), n, 2*r+1, vertical=True) #size is r*n where r is 5
vals = torch.ones(ind.size(0), dtype=torch.float) #number of edges
vals = vals / sum_sparse(ind, vals, size)
ind.size(1)
enrich(data.triples,n,r)

In [ ]:
data = kg.load('micro', torch=True) 
print(f'Number of entities: {data.num_entities}') #data.i2e
print(f'Number of classes: {data.num_classes}')
print(f'Types of relations: {data.num_relations}') #data.i2r
data.triples


### RGCN
- Horizontal indexes are: edge indexes when edge connections stacked horizontally
- Vertical indexes: // vertically
- hor graph is the horizontal adjacency 

### IDEA
Use the stacked adjacecny as it is - but find a way to index back each node to its original - it should just be a matter of dividing by the offset

In [ ]:
idxt, clst = data.training[:, 0], data.training[:, 1]
idxw, clsw = data.withheld[:, 0], data.withheld[:, 1]
idxt, clst = idxt.long(), clst.long()
idxw, clsw = idxw.long(), clsw.long()
data.training.data
data.withheld
data.num_classes #4 Possible values for y
data.training.data.shape
data.withheld


In [ ]:
class RGCN(nn.Module):
    """
    Classic RGCN
    """

    def __init__(self, triples, n, r, numcls, emb=16, bases=None):

        super().__init__()

        self.emb = emb
        self.bases = bases
        self.numcls = numcls

        self.triples = enrich(triples, n, r)

        # horizontally and vertically stacked versions of the adjacency graph
        hor_ind, hor_size = adj(self.triples, n, 2*r+1, vertical=False)
        ver_ind, ver_size = adj(self.triples, n, 2*r+1, vertical=True)
        #number of relations is 2*r+1 because we added the inverse and self loop

        _, rn = hor_size #horizontally stacked adjacency matrix size
        r = rn // n #number of relations enriched divided by number of nodes

        vals = torch.ones(ver_ind.size(0), dtype=torch.float) #number of enriched triples
        vals = vals / sum_sparse(ver_ind, vals, ver_size) #normalize the values by the number of edges

        hor_graph = torch.sparse.FloatTensor(indices=hor_ind.t(), values=vals, size=hor_size) #size: n,r, emb
        
        
        self.register_buffer('hor_graph', hor_graph)

        ver_graph = torch.sparse.FloatTensor(indices=ver_ind.t(), values=vals, size=ver_size)
        self.register_buffer('ver_graph', ver_graph)

        # layer 1 weights
        if bases is None:
            self.weights1 = nn.Parameter(torch.FloatTensor(r, n, emb))
            nn.init.xavier_uniform_(self.weights1, gain=nn.init.calculate_gain('relu'))

            self.bases1 = None
        else:
            self.comps1 = nn.Parameter(torch.FloatTensor(r, bases))
            nn.init.xavier_uniform_(self.comps1, gain=nn.init.calculate_gain('relu'))

            self.bases1 = nn.Parameter(torch.FloatTensor(bases, n, emb))
            nn.init.xavier_uniform_(self.bases1, gain=nn.init.calculate_gain('relu'))

        # layer 2 weights
        if bases is None:

            self.weights2 = nn.Parameter(torch.FloatTensor(r, emb, numcls) )
            nn.init.xavier_uniform_(self.weights2, gain=nn.init.calculate_gain('relu'))

            self.bases2 = None
        else:
            self.comps2 = nn.Parameter(torch.FloatTensor(r, bases))
            nn.init.xavier_uniform_(self.comps2, gain=nn.init.calculate_gain('relu'))

            self.bases2 = nn.Parameter(torch.FloatTensor(bases, emb, numcls))
            nn.init.xavier_uniform_(self.bases2, gain=nn.init.calculate_gain('relu'))

        self.bias1 = nn.Parameter(torch.FloatTensor(emb).zero_())
        self.bias2 = nn.Parameter(torch.FloatTensor(numcls).zero_())

    def forward2(self, hor_graph, ver_graph):


        ## Layer 1

        n, rn = hor_graph.size() #horizontally stacked adjacency matrix size
        r = rn // n
        e = self.emb
        b, c = self.bases, self.numcls

        if self.bases1 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps1, self.bases1)
            weights = torch.mm(self.comps1, self.bases1.view(b, n*e)).view(r, n, e)
        else:
            weights = self.weights1

        assert weights.size() == (r, n, e) #r relations, n nodes, e embedding size

        # Apply weights and sum over relations
        #hidden layer
        h = torch.mm(hor_graph, weights.view(r*n, e))  #matmul with horizontally stacked adjacency matrix and initialized weights
        assert h.size() == (n, e)

        h = F.relu(h + self.bias1) #apply non linearity and add bias

        ## Layer 2

        # Multiply adjacencies by hidden
        h = torch.mm(ver_graph, h) # sparse mm
        h = h.view(r, n, e) # new dim for the relations

        if self.bases2 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps2, self.bases2)
            weights = torch.mm(self.comps2, self.bases2.view(b, e * c)).view(r, e, c)
        else:
            weights = self.weights2

        # Apply weights, sum over relations
        # h = torch.einsum('rhc, rnh -> nc', weights, h)
        h = torch.bmm(h, weights).sum(dim=0)

        assert h.size() == (n, c)

        return h + self.bias2 # -- softmax is applied in the loss
    



    def forward(self):


        ## Layer 1

        n, rn = self.hor_graph.size() #horizontally stacked adjacency matrix size
        r = rn // n
        e = self.emb
        b, c = self.bases, self.numcls

        if self.bases1 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps1, self.bases1)
            weights = torch.mm(self.comps1, self.bases1.view(b, n*e)).view(r, n, e)
        else:
            weights = self.weights1

        assert weights.size() == (r, n, e) #r relations, n nodes, e embedding size

        # Apply weights and sum over relations
        #hidden layer
        h = torch.mm(self.hor_graph, weights.view(r*n, e))  #matmul with horizontally stacked adjacency matrix and initialized weights
        assert h.size() == (n, e)

        h = F.relu(h + self.bias1) #apply non linearity and add bias

        ## Layer 2

        # Multiply adjacencies by hidden
        h = torch.mm(self.ver_graph, h) # sparse mm
        h = h.view(r, n, e) # new dim for the relations

        if self.bases2 is not None:
            # weights = torch.einsum('rb, bij -> rij', self.comps2, self.bases2)
            weights = torch.mm(self.comps2, self.bases2.view(b, e * c)).view(r, e, c)
        else:
            weights = self.weights2

        # Apply weights, sum over relations
        # h = torch.einsum('rhc, rnh -> nc', weights, h)
        h = torch.bmm(h, weights).sum(dim=0)

        assert h.size() == (n, c)

        return h + self.bias2 # -- softmax is applied in the loss

    def penalty(self, p=2):
        """
        L2 penalty on the weights
        """
        assert p==2

        if self.bases is None:
            return self.weights1.pow(2).sum()

        return self.comps1.pow(p).sum() + self.bases1.pow(p).sum()

In [ ]:
def go(name='aifb', lr=0.01, wd=0.0, l2=0.0, epochs=50, prune=False, optimizer='adam', final=True,  emb=16, bases=None, printnorms=True):

    include_val = name in ('aifb','mutag','bgs','am')
    # -- For these datasets, the validation is added to the training for the final eval.

    data = load(name, torch=True, prune_dist=2 if prune else None, final=final, include_val=include_val)

    print(f'{data.triples.size(0)} triples')
    print(f'{data.num_entities} entities')
    print(f'{data.num_relations} relations')

    tic()
    rgcn = RGCN(data.triples, n=data.num_entities, r=data.num_relations, numcls=data.num_classes, emb=emb, bases=bases)

    if torch.cuda.is_available():
        print('Using cuda.')
        rgcn.cuda()

        data.training = data.training.cuda()
        data.withheld = data.withheld.cuda()

    print(f'construct: {toc():.5}s')

    if optimizer == 'adam':
        opt = torch.optim.Adam(lr=lr, weight_decay=wd, params=rgcn.parameters())
    elif optimizer == 'adamw':
        opt = torch.optim.AdamW(lr=lr, weight_decay=wd, params=rgcn.parameters())
    else:
        raise Exception(f'Optimizer {optimizer} not known')

    for e in range(epochs):
        tic()
        opt.zero_grad()
        out = rgcn()
        print('out',out)

        idxt, clst = data.training[:, 0], data.training[:, 1]
        idxw, clsw = data.withheld[:, 0], data.withheld[:, 1]
        idxt, clst = idxt.long(), clst.long()
        idxw, clsw = idxw.long(), clsw.long()
        out_train = out[idxt, :]
        loss = F.cross_entropy(out_train, clst, reduction='mean')
        if l2 != 0.0:
            loss = loss + l2 * rgcn.penalty()

        # compute performance metrics
        with torch.no_grad():
            training_acc = (out[idxt, :].argmax(dim=1) == clst).sum().item() / idxt.size(0)
            withheld_acc = (out[idxw, :].argmax(dim=1) == clsw).sum().item() / idxw.size(0)
            #torch.save(out[idxw, :].argmax(dim=1) , 'aifb_chk/prediction_aifb')
        loss.backward()
        opt.step()

        if printnorms is not None:
            # Print relation norms layer 1
            nr = data.num_relations
            weights = rgcn.weights1 if bases is None else rgcn.comps1

            ctr = Counter()

            for r in range(nr):

                ctr[data.i2r[r]] = weights[r].norm()
                ctr['inv_'+ data.i2r[r]] = weights[r+nr].norm()

            print('relations with largest weight norms in layer 1.')
            for rel, w in ctr.most_common(printnorms):
                print(f'     norm {w:.4} for {rel} ')

            weights = rgcn.weights2 if bases is None else rgcn.comps2

            ctr = Counter()
            for r in range(nr):

                ctr[data.i2r[r]] = weights[r].norm()
                ctr['inv_'+ data.i2r[r]] = weights[r+nr].norm()

            print('relations with largest weight norms in layer 2.')
            for rel, w in ctr.most_common(printnorms):
                print(f'     norm {w:.4} for {rel} ')
        torch.save(out[idxw, :], 'aifb_chk/prediction_aifb')
        torch.save(rgcn,'model_aifb')
        print(f'epoch {e:02}: loss {loss:.2}, train acc {training_acc:.2}, \t withheld acc {withheld_acc:.2} \t ({toc():.5}s)')

In [ ]:
print('arguments ', ' '.join(sys.argv))
fire.Fire(go)

In [ ]:
model = torch.load('model_aifb')
pred = model.forward2(hor_graph, ver_graph)
pred.argmax(dim=1).shape
#data.training[:,1]

### SO
So instead of adjacency matrix and x - we could use for the forward hor_graph /ver_graph - the problem is how to define the neighborhood subgraph???

ALSO the problem is that we cant just go to a matrix from the edges (sparse tensor)- it woukld be too big and computationally expensive!!

Have to find another way to get the neighborsss


through indexes maybeeee


In [ ]:
def get_adjacency(hor_graph):
    adj = torch.zeros(hor_graph.size(1),hor_graph.size(1))
    for edge in hor_graph.coalesce().indices().t():
        adj[edge[0]][edge[1]] = 1
    return adj

In [ ]:
get_adjacency(hor_graph)

In [ ]:
data.withheld[:,0]

In [ ]:
753935/8285 #this is the number of relations 45*2 +1

In [ ]:
for i in data.withheld[:,0]:
    if i in sparse_tensor.indices().t()[:,0]:
        print(sparse_tensor.indices().t()[:,0][i])

# data.withheld[:,0]
# sparse_tensor.indices().t()[:,0]

ChatGPT implementation


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_sparse import SparseTensor

class RGCNLayer(nn.Module):
    def __init__(self, in_features, out_features, num_rels):
        super(RGCNLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.num_rels = num_rels
        self.weight = nn.Parameter(torch.Tensor(self.num_rels, self.in_features, self.out_features))
        nn.init.xavier_uniform_(self.weight, gain=nn.init.calculate_gain('relu'))

    def forward(self, x, edge_index, edge_type):
        # x = data.triples
        # edge_index = data.edge_index.t()
        # edge_type = data.edge_type
        num_nodes = x.size(0)
        num_edges = edge_index.size(1)
        rel_weights = self.weight[edge_type].view(num_edges, self.in_features, self.out_features)
        rel_adj_list = [[] for _ in range(self.num_rels)]
        for i in range(num_edges):
            src = edge_index[0][i].item()
            dst = edge_index[1][i].item()
            rel = edge_type[i].item()
            rel_adj_list[rel].append((src, dst))
        h = x.unsqueeze(1)
        rel_messages = []
        for r in range(self.num_rels):
            if len(rel_adj_list[r]) == 0:
                continue
            rel_adj = torch.LongTensor(rel_adj_list[r]).t()
            adj_t = SparseTensor(row=rel_adj[1], col=rel_adj[0], sparse_sizes=(num_nodes, num_nodes))
            rel_messages.append(adj_t.matmul(h).squeeze())
        rel_messages = torch.cat(rel_messages, dim=1)
        output = torch.bmm(rel_messages.unsqueeze(1), rel_weights).squeeze()
        return F.relu(output)

class RGCN(nn.Module):
    def __init__(self, num_entities, num_relations, hidden_dim, num_classes):
        super(RGCN, self).__init__()
        self.num_entities = num_entities
        self.num_relations = num_relations
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.embedding = nn.Embedding(self.num_entities, self.hidden_dim)
        self.layers = nn.ModuleList()
        self.layers.append(RGCNLayer(self.hidden_dim, self.hidden_dim, self.num_relations))
        self.layers.append(RGCNLayer(self.hidden_dim, self.num_classes, self.num_relations))

    def forward(self, x, edge_index, edge_type):
        x = self.embedding(x)
        for layer in self.layers:
            x = layer(x, edge_index, edge_type)
        return F.softmax(x, dim=1)


In [ ]:
# #data.triples[[data.training[:,0]]]
# data.triples[[data.training[:,0].detach().numpy()]]
# #data.training[:,0].detach().numpy()
# data.edge_index.t()[[data.training[:,0].detach().numpy()]].detach().numpy()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RGCN(data.num_entities, data.num_relations, 16, data.num_classes).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)



optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      #out = model(data.triples[[data.training[:,0].detach().numpy()]].detach().numpy(), data.edge_index.t()[[data.training[:,0].detach().numpy()]].detach().numpy(), data.edge_type()[[data.training[:,0].detach().numpy()]].detach().numpy())  # Perform a single forward pass.
      out = model(data.triples, data.edge_index,data.edge_type)
      loss = criterion(out[data.training[:,0]], data.training[:,1])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

# def test():
#       model.eval()
#       out = model(data.x, get_adjacency(data))
#       pred = out.argmax(dim=1)  # Use the class with highest probability.
#       test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
#       test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
#       torch.save(out, 'cora_chk/prediction_cora')
#       return test_acc


for epoch in tqdm(range(1, 401)):
    loss = train()

# test_acc = test()
# print(f'Test Accuracy: {test_acc:.4f}') 


model.eval()
torch.save(model, 'rel')

# pred, adj = model(data.x, get_adjacency(data))
# pred_label = pred.argmax(dim=1)